<a href="https://colab.research.google.com/github/datagirl98/Capstone2025/blob/main/Capstone_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the drive helper
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler,OneHotEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

In [14]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/healthcare_dataset.csv')
df.head()

Name  Age  Gender Blood Type Medical Condition Date of Admission  \
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31   
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22   
3   andrEw waTtS   28  Female         O+          Diabetes        2020-11-18   
4  adrIENNE bEll   43  Female        AB+            Cancer        2022-09-19   

             Doctor                    Hospital Insurance Provider  \
0     Matthew Smith             Sons and Miller         Blue Cross   
1   Samantha Davies                     Kim Inc           Medicare   
2  Tiffany Mitchell                    Cook PLC              Aetna   
3       Kevin Wells  Hernandez Rogers and Vang,           Medicare   
4    Kathleen Hanna                 White-White              Aetna   

   Billing Amount  Room Number Admission Type Discharge Date   Medication  \
0    18856.281306          328         Urgent     2024-02-02  Paracetamol   
1    33643.327287          265      Emergency     2019-08-26    Ibuprofen   
2    27955.096079          205      Emergency     2022-10-07      Aspirin   
3    37909.782410          450       Elective     2020-12-18    Ibuprofen   
4    14238.317814          458         Urgent     2022-10-09   Penicillin   

   Test Results  
0        Normal  
1  Inconclusive  
2        Normal  
3      Abnormal  
4      Abnormal

In [15]:
# Drop unnecessary columns
df.drop(['Name', 'Room Number', 'Date of Admission', 'Discharge Date'], axis=1, inplace=True)

# Convert billing amount to categories
def billing_category(amount):
    if amount <= 1000:
        return 'Low'
    elif amount <= 5000:
        return 'Medium'
    else:
        return 'High'

df['Billing Category'] = df['Billing Amount'].apply(billing_category)

# Encode categorical variables
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# Prepare features and target
X = df.drop(['Billing Amount', 'Billing Category'], axis=1)
y = df['Billing Category']

In [16]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [18]:
model = XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss',
    random_state=42
)


In [22]:

# Create Billing Categories
def billing_category(amount):
    if amount <= 1000:
        return 'Low'
    elif amount <= 5000:
        return 'Medium'
    else:
        return 'High'

df['Billing Category'] = df['Billing Amount'].apply(billing_category)

# Encode categorical variables
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# Define features and target
X = df.drop(['Billing Amount', 'Billing Category'], axis=1)
y = df['Billing Category']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train XGBoost model
model = XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss',
    random_state=42
)

model.fit(X_train_resampled, y_train_resampled)

# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.92      0.78      0.84     10188
           1       0.01      0.07      0.02        85
           2       0.10      0.24      0.14       827

    accuracy                           0.73     11100
   macro avg       0.34      0.36      0.33     11100
weighted avg       0.85      0.73      0.78     11100

